In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(temperature = 0, model="qwen3-vl:8b")

response = llm.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")
print(response)

content=" Maybe include some examples from different regions.\n\nAlso, note that some trees might have different tolerances based on cultivars or specific varieties.\n\nFor example, the Siberian larch is very cold-hardy, while some maples might be less so depending on the cultivar.\n\nWait, the user is asking for trees that are strong in cold resistance, so the answer should focus on the most cold-hardy species.\n\nLet me structure the answer:\n\n1. Conifers:\n   - Norway spruce (Picea abies): Common in Europe, tolerates -30°C.\n   - White spruce (Picea glauca): Native to North America, can handle -40°C.\n   - Siberian larch (Larix sibirica): Extremely cold-hardy, native to Siberia, tolerates -50°C.\n   - Siberian pine (Pinus sibirica): Native to Siberia, cold-resistant.\n   - Eastern white pine (Pinus strobus): Found in northeastern US, cold-hardy.\n\n2. Deciduous trees:\n   - Sugar maple (Acer saccharum): Tolerates -30°C.\n   - Red maple (Acer rubrum): Cold-hardy, can handle -20°C to

In [66]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
schema_text = db.get_table_info()
tables = db.get_usable_table_names()

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Sample output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [63]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [64]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("""
You are a SQL expert.

Dialect: {dialect}
Schema:
{schema}

Question:
{question}

Write a correct SQL query.
Return ONLY the SQL query.
""")

chain = prompt | llm | StrOutputParser()

In [65]:
sql = chain.invoke({
    "dialect": "SQLite",
    "schema": schema_text,
    "question": "emplyoee 몇명인지 말해줘",
})

print(sql)

.
</think>

SELECT COUNT(*) FROM Employee;
